In [8]:
%matplotlib
from cartopy import crs, feature
import ESMF
from matplotlib import cm, gridspec, rcParams
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset

import pyresample

def extract_pixel_coordinates(ULx,Uly,LRx,LRy,shape):
    in_proj  = Proj('+proj=sinu +R=6371007.181 +nadgrids=@null +wktext')
    out_proj = Proj(init='epsg:4326')
    x        = np.linspace(ULx, LRx, shape[0], endpoint=False) + abs((ULx-LRx)/(2*shape[0]))
    y        = np.linspace(ULy, LRy, shape[0], endpoint=False) - abs((ULy-LRy)/(2*shape[0]))
    xx, yy   = np.meshgrid(x,y)
    plon, plat = transform(in_proj, out_proj, xx, yy)
    mlon, mlat = plon[0], np.array([i[0] for i in plat])
    plon       = plon.flatten()
    plat       = plat.flatten()
    return plon, plat, mlon, mlat
ESMF.Manager(debug = True);
FORTRAN_CONTIGUOUS = True;
#shape              = (200 , 200);

Using matplotlib backend: Qt5Agg


In [9]:
# READ REFLECTANCE FILE AND EXTRACT REFLECTANCE, LATITUDES & LONGITUDES
fc       = Dataset('MCD43A4/121/MCD43A4.A2018121.h10v05.006.2018130033209.hdf', mode='r')
struct   = getattr(fc, 'StructMetadata.0')
struct1  = struct[struct.find('UpperLeftPointMtrs'): struct.find('LowerRightMtrs')][19:-3]
struct2  = struct[struct.find('LowerRightMtrs')    : struct.find('Projection')    ][15:-3]
ULx, ULy = literal_eval(struct1)
LRx, LRy = literal_eval(struct2)
fc_lon, fc_lat, fc_mlon, fc_mlat  = extract_pixel_coordinates(ULx,ULy,LRx,LRy,(2400,2400))
lon_curv, lat_curv = np.meshgrid(fc_mlon, fc_mlat)

nrb = fc.variables['Nadir_Reflectance_Band1'][:]
fv  = fc.variables['Nadir_Reflectance_Band1']._FillValue
lat1 = np.linspace(fc_mlat[0], fc_mlat[-1], 1200, endpoint=True)# + abs((fc_mlat[0]-fc_mlat[-1])/(2*200))
lon1 = np.linspace(fc_mlon[0], fc_mlon[-1], 1200, endpoint=True)# - abs((fc_mlon[0]-fc_mlon[-1])/(2*200))
lon2d, lat2d = np.meshgrid(lon1, lat1)

In [10]:
targ_def = pyresample.geometry.SwathDefinition(lons=lon2d, lats=lat2d)
orig_def = pyresample.geometry.SwathDefinition(lons=lon_curv, lats=lat_curv)

wf = lambda r: 1/r**2
nrb_idw = pyresample.kd_tree.resample_custom(orig_def, nrb, targ_def, radius_of_influence=500000, neighbours=4, weight_funcs=wf, fill_value=32767)

TypeError: issubclass() arg 2 must be a class or tuple of classes

In [ ]:
# CREATE SOURCE GRID & FIELD FOR PYRESAMPLE
source_lons, source_lats = np.meshgrid(ref_mlon, ref_mlat)
orig_def                 = pyresample.geometry.SwathDefinition(lons=source_lons, lats=source_lats)

# CREATE DESTINATION GRID & FIELD FOR PYRESAMPLE
t_lat                    = np.linspace(ref_mlat[0], ref_mlat[-1], shape[0], endpoint=True)
t_lon                    = np.linspace(ref_mlon[0], ref_mlon[-1], shape[1], endpoint=True)                         
dest_lons, dest_lats     = np.meshgrid(t_lon, t_lat)
dest_def                 = pyresample.geometry.SwathDefinition(lons=dest_lons, lats=dest_lats)

nrb1a       = nrb1a.flatten()
wf          = lambda r: 1/r**2
nrb1b       = pyresample.kd_tree.resample_custom(orig_def, nrb1a, dest_def, radius_of_influence=500000, neighbours=8, weight_funcs=wf, fill_value=float(fv1))

In [ ]:
sourcegrid = ESMF.Grid(np.array(nam_shape), staggerloc=ESMF.StaggerLoc.CENTER, coord_sys=ESMF.CoordSys.SPH_DEG)
source_lon = sourcegrid.get_coords(0)
source_lat = sourcegrid.get_coords(1)

if FORTRAN_CONTIGUOUS:
    source_lon[...] = nam_lon.T
    source_lat[...] = nam_lat.T
else:
    source_lon[...] = nam_lon
    source_lat[...] = nam_lat

sourcefield = ESMF.Field(sourcegrid, name='Ref_Orig')

In [ ]:
sourcegrid

In [ ]:
sif_lat  = np.array([30.05, 38.23, 30.87, 30.01, 38.88])
sif_lon  = np.array([-81, -77, -90.54, -88.4, -79.56])

In [ ]:
sif_loc  = ESMF.LocStream(len(sif_lat), coord_sys = ESMF.CoordSys.SPH_DEG, name = 'sif_locs')

sif_data = ESMF.Field(sif_loc, name = 'Town 2 m NAM Dewpoint')

regrid   = ESMF.Regrid(sourcefield,sif_data, regrid_method = ESMF.RegridMethod.BILINEAR)#, unmapped_action = ESMF.UnmappedAction.IGNORE)

